In [ ]:
library(DBI)
library(dbplyr)
library(bigrquery)
suppressPackageStartupMessages(library(tidyverse))
library(lubridate)

In [ ]:
options(tibble.width = Inf
# The page size was adjusted as bigquery was throwing errors
con <- DBI::dbConnect(bigquery(), project = "yhcr-prd-phm-bia-core", bigint = "integer64", page_size = 15000)

In [ ]:
db <- 'CB_STAGING_DATABASE_YAS_FDM_Format'
gp_db <- 'CB_FDM_PrimaryCare'

start_date = '2022-01-01'
end_date = '2023-07-01'

# Retrieve IUC data

In [ ]:
iuc_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{db}.tbl_YAS_IUC"))

In [ ]:
iuc_tbl %>% glimpse()

In [ ]:
iuc_tbl2 <- iuc_tbl %>%
    #head() %>%
    select(
        CaseRef,
        person_id,
        Call_Commenced_Date_Time,
        tbl_YAS_IUC_start_date,
        Call_Handler_1_Triage_Start_Time, 
        Call_Handler_2_Triage_Start_Time, 
        Call_Handler_3_Triage_Start_Time, 
        Clinical_Advisor_1_Triage_Start_Time, 
        Clinical_Advisor_2_Triage_Start_Time, 
        Clinical_Advisor_3_Triage_Start_Time,
        Highest_Skillset,
        Age,
        Sex,
        GP_Surgery_ID,
        Final_Symptom_Group,
        Final_Symptom_Discriminator,
        Final_Dx_Code,
        Final_Dx_Description,
        Referral_Service,
        Referral_Service_Type,
        Rejected_Service,
        Rejected_Service_Reason,
    ) %>%
    filter(
        tbl_YAS_IUC_start_date >= start_date,
        tbl_YAS_IUC_start_date < end_date
    ) %>%
    distinct()
        

In [ ]:
iuc_tbl2 %>% glimpse()

In [ ]:
iuc_tbl2 %>% summarise(thedate = max(tbl_YAS_IUC_start_date)) 

In [ ]:
iuc_tbl2 %>% count() # 1529576 # v2 2100112

## LSOA IMD 2019 CSV

In [ ]:
# The first time this code is run, you'll need to uncomment this code chunk and the next one.

#lsoa_imd_2019_df <- read_csv('data/imd2019lsoa.csv')

In [ ]:
# lsoa_imd_2019_df2 <- lsoa_imd_2019_df %>%
#     group_by(FeatureCode) %>%
#     summarise(
#      imd_rank = first(Value[Measurement == "Rank"]),
#      imd_decile = first(Value[Measurement == "Decile"])
#     ) %>% ungroup() %>% distinct() %>%
#     rename(
#      lsoa = FeatureCode
#     )
#  #Create table for master joining later
# DBI::dbWriteTable(con, "yhcr-prd-phm-bia-core.CB_2121.lsoa_csv_tbl", lsoa_imd_2019_df2)

In [ ]:
#lsoa_imd_2019_df2 %>% glimpse()

In [ ]:
csv_lsoa_tbl <- tbl(con, "yhcr-prd-phm-bia-core.CB_2121.lsoa_csv_tbl")

In [ ]:
csv_lsoa_tbl %>% glimpse()

# Retrieve Patients with Bradford GP

In [ ]:
# Note no date restriction on this query
gp_person_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{gp_db}.person")) %>% #glimpse()
    distinct(person_id, ethnicity_source_value, gender_source_value, birth_datetime)

In [ ]:
gp_person_tbl %>% count() # 1177931 v2 same number

# Assign LSOA to patients

In [ ]:
# Couple of tables required for LSOA and IMD lookup
id_lsoa_tbl <-  tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_person_lsoa"))

In [ ]:
id_lsoa_tbl %>% count() # 1621481 v2 same

In [ ]:
person_lsoa_imd_tbl <- id_lsoa_tbl  %>% left_join(csv_lsoa_tbl, by="lsoa")

In [ ]:
person_lsoa_imd_tbl %>% glimpse()

In [ ]:
person_lsoa_imd_tbl %>% count(imd_decile) %>% print(n=11)

In [ ]:
person_lsoa_tbl <- gp_person_tbl %>%
    left_join(person_lsoa_imd_tbl, by="person_id") 

In [ ]:
person_lsoa_tbl %>% count() # 1210393 v2 same

In [ ]:
person_lsoa_tbl %>% glimpse()

In [ ]:
person_lsoa_tbl %>% collect() %>% count(!is.na(lsoa), is.na(imd_decile))

In [ ]:
iuc_tbl3 <- iuc_tbl2 %>% 
    inner_join(person_lsoa_tbl, by="person_id") 

In [ ]:
iuc_tbl2 %>% count() # 1529576 v2 2100112
iuc_tbl3 %>% count() # 216936  v2 298298

In [ ]:
iuc_tbl3 %>% glimpse()

In [ ]:
iuc_df <- iuc_tbl3 %>% collect() 

In [ ]:
iuc_df %>% glimpse()

In [ ]:
#iuc_df %>% filter(between(tbl_YAS_IUC_start_date, ymd("2023-02-01"), ymd("2023-02-07"))) %>% head()

# Symptom group and discriminator lookup

In [ ]:
wrangle_dt <- function(dt_str) {
 dt_str %>% map_chr(\(dt) if_else(nchar(dt) == 16, paste0(dt, ":00"), dt))   
}

In [ ]:
iuc_df2 <- iuc_df %>% #head() %>%
  distinct(CaseRef, .keep_all = T) %>% # To be pragmatic, we'll only keep one row per CaseRef
  rename_with(tolower, everything()) %>%
  mutate(
    sex = case_when(
         sex == "M" ~ "male",
         sex == "F" ~ "female",
         TRUE ~ NA_character_
      ),
     age = as.numeric(age)
      # Added filter for adults only
  ) %>% filter(!is.na(age) & age >= 18) %>%
  mutate(
    # Some date times have a second value, some don't, sigh.
    across(ends_with('_time'), wrangle_dt),
    across(ends_with('_time'), dmy_hms)
  ) %>%
  rowwise() %>%
  mutate(
    exit_111_date_time = max(c_across(ends_with('_time')), na.rm = T),
    # Matches on column names that start with Clinical_Advisor, has anything in the middle and ends with Time
    clinical_advisor = ifelse(highest_skillset == "Clinical Advisor", TRUE, FALSE),
  ) %>%
  ungroup() %>%
  select(person_id, age, sex, gp_surgery_id, call_commenced_date_time, final_dx_code, final_symptom_group, final_symptom_discriminator, exit_111_date_time, clinical_advisor, ethnicity_source_value,
         rejected_service,
         rejected_service_reason, referral_service, referral_service_type, lsoa, imd_rank, imd_decile) %>%
  distinct()

In [ ]:
iuc_df2 %>% glimpse()

In [ ]:
iuc_df2 %>% count() # 193526 v2 201232 - but excludes children now.

In [ ]:
sg_df <- read_csv('data/symptom_groups.csv')
sd_df <- read_csv('data/symptom_discriminators.csv')

iuc_df3 <- iuc_df2 %>% #head() %>%
    mutate(
        final_symptom_group = as.numeric(final_symptom_group),
        final_symptom_discriminator = as.numeric(final_symptom_discriminator),
        start = call_commenced_date_time,
        end = exit_111_date_time
    ) %>%
    left_join(sg_df, by=c("final_symptom_group"="id")) %>%
    rename(symtom_group_desc = description) %>%
    left_join(sd_df, by=c("final_symptom_discriminator"="id")) %>%
    rename(symptom_discriminator_desc = description) %>%
    mutate(
        unique_id = glue::glue("IUC_{row_number()}")
    )

In [ ]:
iuc_df3 %>% count() # 193526 v2 201323

In [ ]:
iuc_df3 %>% glimpse()

In [ ]:
saveRDS(iuc_df3, 'data/iuc_df.rds')

# Abandoned call data

In [ ]:
ac_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{db}.cb_abandoned_calls"))

In [ ]:
ac_tbl2 <- ac_tbl %>% inner_join(person_lsoa_tbl, by="person_id")

In [ ]:
ac_df <- ac_tbl2 %>% collect() %>% filter(!is.na(person_id)) # Warning: filtering NAs before collection does not appear to work

In [ ]:
ac_df %>% glimpse()

In [ ]:
ac_df %>% count() # 37476 v2 same

In [ ]:
ac_df2 <- ac_df %>% #head() %>%
    transmute(
        person_id,
        call_id = as.numeric(ID),
        start = as_datetime(as.numeric(segstart_utc)),
        end = as_datetime(as.numeric(segstop_utc)),
        queuetime = as.numeric(queuetime),
        ringtime = as.numeric(ringtime),
        lsoa,
        ethnicity_source_value,
        age = as.integer(difftime(start, birth_datetime, unit= "weeks")/52),
        unique_id = glue::glue("ABN_{row_number()}"),
        imd_rank,
        imd_decile,
        sex = case_when(
            gender_source_value %in% c("1", "M", "Male") ~ "male",
            gender_source_value %in% c("2", "F", "Female") ~ "female",
            TRUE ~ NA_character_
        ) 
    ) %>% 
    # There's some duplication in the AC records. Remove duplicates that are from the same person with the same start and end call times
    distinct(person_id, start, end, .keep_all = T) %>%
    filter(!is.na(age) & age >= 18)

In [ ]:
ac_df2 %>% count() # 36245 v2 24304 (excluding children)

In [ ]:
ac_df2 %>% glimpse()

In [ ]:
ac_df2 %>% filter(!is.na(lsoa), is.na(imd_rank)) %>% count() # 1098 with no IMD, but with an LSOA v2 same

In [ ]:
ac_df2 %>% count(n_distinct(person_id)) # 25489, 36245 v2 16693, 24304

In [ ]:
index_event_fn <- function(person_id, row_num, end) {
  #print(row_num)
  df <- tibble(person_id = person_id, row_num = row_num, end = end, index_ref = NA_integer_) %>%
    arrange(end)
  
  if(nrow(df) == 1) {
    # Easy, only one record so must be index call
    df <- df %>% 
      mutate(
        prev_row_num = row_num,
        index_event = 1,
        index_ref = NA_integer_
    )
  } else {
    
    df <- df %>% 
      mutate(
        #index_event = 1,
        index_event = case_when(
          row_number() == 1 ~ 1,
          as.numeric(difftime(end, lag(end), units = 'hours')) > 72 ~ 1,
          TRUE ~ 0
        )
      )
    
    # Bit hacky but couldn't work out another way to capture the index event row number
    # and then replicate this through subsequent calls that occured within 72 hours
    # and so were linked to the index event
    index_ref_num = 0
    for (row in 1:nrow(df)) {
      #print(row)
      if(df$index_event[row] == 1) {
       # print('index row')
        index_ref_num = df$row_num[row]
        df$index_ref[row] = NA_integer_
      } else {
        df$index_ref[row] = index_ref_num
      }
      
    }
    
  }
  
  return(df %>% select(index_event, index_ref))
}

In [ ]:
ac_df3 <- ac_df2 %>% 
    arrange(person_id, end) %>%
    mutate(
        row_num = row_number()
    ) %>%
    group_by(person_id) %>%
    mutate(
     index_event_fn(person_id, row_num, end)   
    ) %>%
    ungroup()

In [ ]:
ac_df3 %>% count() # 36245 v2 24304

In [ ]:
ac_df4 <- ac_df3 %>% filter(index_event == 1)

In [ ]:
ac_df4 %>% count() # 33332 v2 22270

In [ ]:
saveRDS(ac_df3, 'data/ac_df_all.rds')
saveRDS(ac_df4, 'data/ac_df_index.rds')

# CAD data

In [ ]:
cad_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{db}.tbl_YAS_999"))
cad_tbl %>% glimpse()

In [ ]:
cad_tbl %>% count() # 2491216 v2 same

In [ ]:
# Need to exclude things like the daily log
resource_types <- c("Emergency Ambulance", "Rapid Response", "Low Acuity Transport", "HART USAR 4X4", "HART Heavy Equipment","Patient Transport Vehicle","Specialist Paramedic", "Mental Health Vehicle", "Clinician Remote Worker")

# At the moment, the end_date is the same as start_date, so need to some stuff later to get the correct end date
cad_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{db}.tbl_YAS_999")) %>%
    filter(ResourceType %in% resource_types) %>% #glimpse()
    select(
        person_id, tbl_YAS_999_start_date, TimeClear, CallNumber, MainPatientAge, MainPatientSex, ChiefComplaint, DespatchCode, GovtStdTOC
    ) %>%
    filter(
        tbl_YAS_999_start_date >= start_date,
        tbl_YAS_999_start_date < end_date
    ) %>%
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct(CallNumber, .keep_all = T)

In [ ]:
cad_df <- cad_tbl %>% collect() %>% mutate(
        unique_id = glue::glue("CAD_{row_number()}")
    )

In [ ]:
cad_df %>% count() # 74,611 v2 same

In [ ]:
test_df <- cad_df %>% #head() %>%
    transmute( 
        MainPatientAge,
        age = as.integer(difftime(tbl_YAS_999_start_date, birth_datetime, unit = "weeks")/52),
        MainPatientSex,
        gender_source_value,
        gender = case_when(
         gender_source_value %in% c("1", "Male", "M") ~ "male",
         gender_source_value %in% c("2", "Female", "F") ~ "female",
         TRUE ~ NA_character_
        ),
        main_sex = case_when(
         MainPatientSex == "Male" ~ "male",
         MainPatientSex == "Female" ~ "female",
         TRUE ~ NA_character_
        ),
        sex = coalesce(gender, main_sex)
    )

In [ ]:
test_df %>% count(is.na(age), is.na(MainPatientAge))

In [ ]:
test_df %>% count(is.na(sex))

In [ ]:
test_df %>% count(MainPatientSex)

In [ ]:
# Had a bit of a mare with case_when...had to separate out variable creation as it failed in the next_day/same_day 
# criteria when left in situ
cad_df1 <- cad_df %>% #head(n=500) %>%
    #rowwise() %>%
    transmute(
    person_id,
    unique_id,
    start = tbl_YAS_999_start_date,
    next_day = ymd_hms(glue::glue("{as.Date(tbl_YAS_999_start_date + days(1))} {TimeClear}")),
    same_day = ymd_hms(glue::glue("{as.Date(tbl_YAS_999_start_date)} {TimeClear}")),
    bool_next_day = if_else(as.integer(hour(tbl_YAS_999_start_date)) > as.integer(str_sub(TimeClear, 1, 2)), TRUE, FALSE),
    end = case_when(
          nchar(TimeClear) == 19 & (str_sub(TimeClear, 3,3) == '/') ~ dmy_hms(TimeClear),
          nchar(TimeClear) == 19 & (str_sub(TimeClear, 3,3) != '/') ~ ymd_hms(TimeClear),
          bool_next_day == TRUE ~ next_day,
          bool_next_day == FALSE ~ same_day,
         TRUE ~ NA_POSIXct_
     ),
    imd_rank,
    ethnicity_source_value,
    imd_decile,
    gender = case_when(
     gender_source_value %in% c("1", "Male", "M") ~ "male",
     gender_source_value %in% c("2", "Female", "F") ~ "female",
     TRUE ~ NA_character_
    ),
    main_sex = case_when(
     MainPatientSex == "Male" ~ "male",
     MainPatientSex == "Female" ~ "female",
     TRUE ~ NA_character_
    ),
    sex = coalesce(gender, main_sex),
    age = as.integer(difftime(tbl_YAS_999_start_date, birth_datetime, unit = "weeks")/52),
    GovtStdTOC,
    ChiefComplaint,
    DespatchCode
    ) %>% select(-gender, -main_sex)

In [ ]:
cad_df1  %>% head() # filter(is.na(tbl_YAS_999_end_date))

In [ ]:
cad_df1 <- cad_df1 %>%
  mutate(
    # In the event that there is no end time, assuming incident takes approx 2 hours to complete
   end = if_else(end < start, start + hours(2), end)   
  )

In [ ]:
saveRDS(cad_df1, 'data/cad_df.rds')

# GP data

In [ ]:
# Set of codes I curated from OpenCodeList that denote a clinician-patient consultation
gp_codes_df <- read_csv("data/richard_pilbery-primary-care-clinician-patient-interaction-72708505.csv")#

dbWriteTable(con, glue::glue("yhcr-prd-phm-bia-core.CB_2121.gp_codes"), gp_codes_df, overwrite = T) 

gp_codes_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_2121.gp_codes"))

In [ ]:
gp_codes_tbl %>% glimpse()

In [ ]:
gp_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{gp_db}.tbl_srcode")) %>% glimpse()

In [ ]:
gp_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.{gp_db}.tbl_srcode")) %>% #glimpse()
  select(
    person_id,
    tbl_srcode_start_date,
    tbl_srcode_end_date,
    ctv3code,
    ctv3text,
    idevent,
    gp_surgery_id = idorganisationvisibleto
  ) %>%
  filter(
    tbl_srcode_start_date >= start_date,
    tbl_srcode_end_date < end_date
  ) %>% distinct() %>%
  inner_join(
    gp_codes_tbl, by=c("ctv3code"="code")
  ) %>% distinct() %>%
  left_join(person_lsoa_tbl, by="person_id") 

In [ ]:
gp_tbl %>% glimpse()

In [ ]:
gp_df <- gp_tbl %>% collect(page_size = 25000) # page_size to avoid bigquery errors

In [ ]:
gp_df %>% count() # 2505078 v2 same

In [ ]:
gp_df2 <- gp_df %>%
  group_by(idevent) %>% 
  mutate (
    ctv = paste0(term, collapse = ", ")
  ) %>% 
  ungroup() %>%
  select(-ctv3code, -ctv3text, -term) %>%
  distinct(idevent, .keep_all = T) %>%
  transmute(
      person_id,
      gp_surgery_id,
      start = tbl_srcode_start_date,
      # Make consultation 10 mins if end time = start time
      end = if_else(tbl_srcode_start_date == tbl_srcode_end_date, tbl_srcode_end_date + minutes(10), tbl_srcode_end_date),
      ctv,
      unique_id = glue::glue("GP_{row_number()}"),
      sex = case_when(
         gender_source_value %in% c("1", "Male", "M") ~ "male",
         gender_source_value %in% c("2", "Female", "F") ~ "female",
         TRUE ~ NA_character_
        ),
      age = as.integer(difftime(tbl_srcode_start_date, birth_datetime, unit = "weeks")/52),
      lsoa,
      imd_decile,
      imd_rank,
      ethnicity_source_value
    )

In [ ]:
gp_df2 %>% glimpse()

In [ ]:
saveRDS(gp_df2, 'data/gp_df.rds')

# Hospital data

Avoidable ED attendance HRG  IN ('VB07Z', 'VB08Z', 'VB09Z', 'VB11Z') AND not admitted AND not referred to another healthcare specialist AND did not die : from https://bmjopen.bmj.com/content/bmjopen/10/3/e032043.full.pdf

O'Keeffe definition:

FROM ARCYH UEC non urgent algorithm document:
  --non urgent attendances are a first attendance (attendance category 01) 
to a type1 ED department (department type 01)

AND not investigated in ED (except by urinalysis, pregnancy test or dental investigation) ECDS codes are (1088291000000101 or blank, 27171005, 167252002 / 67900009, 53115007) (investigation)
CDS10 codes are 24,06,21,22, blank

AND Not treated in ED (except by prescription, recording vital signs, dental treatment or guidance/advice) (treatment)
(266712008,413334001,81733005, not applicable, blank)
(07,22,30,56,99,blank)

--AND Discharged completely from care in ED or referred to their GP (attendance disposal)
(989501000000106,1066321000000107, 1066301000000103, 1066311000000101,182992009/3780001  )
(02,03,12)



## BRI in-patient data

In [ ]:
bri_ip_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_FDM_Warehouse_V3.tbl_episode")) %>% #glimpse()
    select(person_id, tbl_episode_start_date, tbl_episode_end_date, diagnoses, diagnosis_1, procedures, procedure_1, gender, birth_year, birth_month) %>%
    rename(
        hosp_sex = gender
    )

In [ ]:
bri_ip_tbl %>% glimpse()

In [ ]:
bri_ip_tbl2 <- bri_ip_tbl %>%
    filter(
        tbl_episode_start_date >= start_date,
        tbl_episode_end_date < end_date
    ) %>% 
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
bri_ip_tbl2 %>% count() # 201365 vs 211727

In [ ]:
bri_ip_df <- bri_ip_tbl2 %>% collect()

In [ ]:
bri_ip_df %>% glimpse()

In [ ]:
saveRDS(bri_ip_df, 'data/bri_ip_df.rds')

## BRI ED data

In [ ]:
bri_ae_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_FDM_Warehouse_V3.tbl_ae")) %>% #glimpse()
    select(
        person_id, 
        hosp_age = age_at_attendance_date,
        hosp_sex = gender,
        tbl_ae_start_date, 
        tbl_ae_end_date, 
        department_type,
        identifier, 
        arrival_mode_description, 
        attendance_disposal,
        attendance_disposal_description, 
        hrg_code, 
        starts_with('investigation'), 
        starts_with('treatment'), 
        starts_with('diagnosis')
    )

In [ ]:
bri_ae_tbl2 <- bri_ae_tbl %>%
    filter(
        tbl_ae_start_date >= start_date,
        tbl_ae_end_date < end_date
    ) %>% 
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
bri_ae_df <- bri_ae_tbl2 %>% collect()

In [ ]:
bri_ae_df %>% count() # 163319 v2 same

In [ ]:
bri_ae_df %>% count(attendance_disposal_description, sort = T)

In [ ]:
# Update to Original O'Keeffee criteria:

# ED Treatment complete  81370   Include as potentially avoidable 
# Admitted as inpatient 42752    **  Not avoidable **
# Streamed to GP / primary care 11285   Include as potentially avoidable 
# Left after assessment before treatment 6333    Include as potentially avoidable 
# Left before initial assessment 4090  Include as potentially avoidable 
# Left after assessment other ED 3367   Include as potentially avoidable
# Streamed to Urgent Care Centre 789  Include as potentially avoidable 
# Streamed to ophthalmology service 153   Include as potentially avoidable 
# Died in the Emergency Care facility 130 **  Not avoidable **
# Streamed to Amb Care service 118 Include as potentially avoidable  - REFUTED  ** NON-AVOIDABLE **
# System Generated 118  ??
# Streamed to mental health service 43  Include as potentially avoidable
# Streamed to Emergency Department 35  ?? I guess this is for cases like seeing children at STH - probably ** not avoidable **
# Streamed to dental service 17  Include as potentially avoidable
# Dead on Arrival 13  ** Not avoidable **
# Discharged with Consent 5  Include as potentially avoidable
# Streamed to pharmacy service 4  Include as potentially avoidable 
# Streamed to falls service 3  Include as potentially avoidable - REFUTED ** NON-AVOIDABLE **
# Streamed to frailty service 1  Include as potentially avoidable - REFUTED ** NON-AVOIDABLE **

In [ ]:
#avoidable_hrg_codes <- c('VB07Z', 'VB08Z', 'VB09Z', 'VB11Z')
non_avoidable_disposal <- c('Admitted as inpatient', 'Died in the Emergency Care facility', 'Streamed to Emergency Department', 'Dead on Arrival', 'Streamed to Amb Care service', 'Streamed to falls service', 'Streamed to frailty service')
#bri_ae_df %>% count(attendance_disposal_description, sort = T)
avoidable_investigation_codes <- c('24', 'NULL', '06', '21', '22')
avoidable_treatment_codes <- c('07', '22', '30', '56', '57', '99', 'NULL')
avoidable_disposal_codes <- c('02', '03', '12')

In [ ]:
bri_ae_df %>% count() #163319 v2 same


In [ ]:
bri_ae_df1 <- bri_ae_df %>% #head(50) %>%
    mutate(
        avoid_invest = case_when(
            #if_all(starts_with('investigation'), ~ .x == 'NULL') ~ 1,
            if_all(starts_with('investigation'), ~ .x %in% avoidable_investigation_codes) ~ 1,
            TRUE ~ 0
        ),
        avoid_treat = case_when(
            if_all(starts_with('treatment'), ~ .x %in% avoidable_treatment_codes) ~ 1,
            TRUE ~ 0
        ),
        avoid_disp = case_when(
            # Based on Sue Croft consult, using revised version
            #attendance_disposal %in% avoidable_disposal_codes ~ 1,
            !attendance_disposal_description %in% non_avoidable_disposal ~ 1,
            TRUE ~ 0
        ),
        avoidable_admission = if_else(avoid_invest == 1 & avoid_treat == 1 & avoid_disp == 1, 1, 0)
    )

In [ ]:
bri_ae_df1 %>% count(avoidable_admission) # 1 - 30068 v2 1 - 27689

In [ ]:
bri_ae_df2 <- bri_ae_df1 %>%
    select(-starts_with('investigation'), -starts_with('treatment'), -starts_with('diagnosis'), -identifier, -department_type) %>%
    distinct()

In [ ]:
bri_ae_df2 %>% glimpse()

In [ ]:
saveRDS(bri_ae_df2, 'data/bri_ae_df.rds')

## Airedale IP data

In [ ]:
airedale_ip_tbla <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20200701_to_20220710_mrg")) %>% #glimpse()
    transmute(
        person_id,
        start = tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_start_date,
        end = tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_end_date,
        hosp_sex = sex,
        dob
   )

airedale_ip_tblb <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20220601_to_Current")) %>% #glimpse()
    transmute(
        person_id,
        start = tbl_SUS_Airedale_APC_20220601_to_Current_start_date,
        end = tbl_SUS_Airedale_APC_20220601_to_Current_end_date,
        hosp_sex = sex,
        dob
   )

In [ ]:
airedale_ip_tbl <- airedale_ip_tbla %>% union_all(airedale_ip_tblb) %>% distinct()

In [ ]:
airedale_ip_tbl %>% count() # 210528 v2 231550

In [ ]:
airedale_ip_tbl2 <- airedale_ip_tbl %>%
    filter(
        start >= start_date,
        end < end_date
    ) %>% 
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
airedale_ip_tbl2 %>% count() # 83566 v2 same

In [ ]:
airedale_ip_df <- airedale_ip_tbl2 %>% collect()

In [ ]:
saveRDS(airedale_ip_df, 'data/airedale_ip_df.rds')

## Airedale ED data

In [ ]:
airedale_ed_tbla <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_ECDS_20150401_to_20220710_mrg")) %>% #glimpse()
    transmute(
        person_id,
        start = tbl_SUS_Airedale_ECDS_20150401_to_20220710_mrg_start_date,
        end = tbl_SUS_Airedale_ECDS_20150401_to_20220710_mrg_end_date,
        disch_dest_snmdct,
        disch_followup_snmdct,
        invest_group,
        treat_group,
        category,
        hosp_sex = sex,
        dateb,
        dob
   )

airedale_ed_tblb <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_ECDS_20220601_to_Current")) %>% #glimpse()
    transmute(
        person_id,
        start = tbl_SUS_Airedale_ECDS_20220601_to_Current_start_date,
        end = tbl_SUS_Airedale_ECDS_20220601_to_Current_end_date,
        disch_dest_snmdct,
        disch_followup_snmdct,
        invest_group,
        treat_group,
        category,
        hosp_age = age_at_cds,
        hosp_sex = sex
    )

In [ ]:
airedale_ed_tbl <- airedale_ed_tbla %>% union_all(airedale_ed_tblb) %>% distinct()

In [ ]:
airedale_ed_tbl2 <- airedale_ed_tbl %>%
    filter(
        start >= start_date,
        end < end_date
    ) %>% 
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
airedale_ed_tbl2 %>% count() # 71373 v2 76566

In [ ]:
airedale_ed_df <- airedale_ed_tbl2 %>% collect()

In [ ]:
airedale_ed_df %>% glimpse()

In [ ]:
airedale_ed_df %>% select(invest_group) %>% head()

In [ ]:
airedale_ed_df2 <- airedale_ed_df %>% #head() %>%
    mutate(
        avoidable_admission = if_else(
            category == "1" & (
                (as.numeric(disch_dest_snmdct) %in% c(989501000000106,1066321000000107, 1066301000000103, 1066311000000101,182992009,3780001) |
                as.numeric(disch_followup_snmdct) %in% c(989501000000106,1066321000000107, 1066301000000103, 1066311000000101,182992009,3780001)
                 ) &
                (grepl("1088291000000101|27171005|167252002|67900009|53115007", invest_group) | is.na(invest_group)) &
                (grepl("266712008|413334001|81733005", treat_group) | is.na(treat_group))
            ), 1, 0)
    )

In [ ]:
airedale_ed_df2 %>% count(avoidable_admission) # v2 1 - 18993

In [ ]:
airedale_ed_df2 %>% glimpse()

In [ ]:
saveRDS(airedale_ed_df2, 'data/airedale_ed_df.rds')

## Calderdale IP data

In [ ]:
calderdale_ip_tbl <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_MYSPACE_RP_CalderdaleSUS.src_SUS_Calderdale_APC_CM_20220210_to_20231116_Part1")) %>% #glimpse()
    transmute(
        person_id,
        hosp_sex = sex,
        dob = NA,
        # Times are all null
        start = PARSE_DATE('%Y%m%d',start_date_hospital_provider_spell),
        end = PARSE_DATE('%Y%m%d',discharge_date_from_hospital_provider_spell)
    ) 

In [ ]:
calderdale_ip_tbl %>% glimpse()

In [ ]:
calderdale_ip_tbl %>%
mutate(
    themonth = LAST_DAY(start)
) %>%
filter(!is.na(person_id)) %>%
count(themonth) %>%
arrange(themonth) %>% print(n=20)

In [ ]:
calderdale_ip_tbl2 <- calderdale_ip_tbl %>%
    filter(
        start >= PARSE_DATE('%Y-%m-%d',start_date),
        end < PARSE_DATE('%Y-%m-%d',end_date)
    ) %>%
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
calderdale_ip_tbl2 %>% count() # 9684 v2 same

In [ ]:
# Need Jan up to mid-Feb 2022

In [ ]:
calderdale_ip_tbl3 <- tbl(con, glue::glue("yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Calderdale_FDM_Format.tbl_SUS_Calderdale_APC_CM_20170618_To_20220731")) %>% #glimpse()
    transmute(
        person_id,
        hosp_sex = sex,
        hosp_age = age_at_start_of_episode,
        # Times are all null
        start = PARSE_DATE('%Y%m%d',start_date_hospital_provider_spell),
        end = PARSE_DATE('%Y%m%d',discharge_date_from_hospital_provider_spell)
    ) 

In [ ]:
calderdale_ip_tbl3 %>% glimpse()

In [ ]:
calderdale_ip_tbl3  %>%
mutate(
    themonth = LAST_DAY(start)
) %>%
filter(!is.na(person_id)) %>%
count(themonth) %>%
arrange(desc(themonth)) %>% print(n=20)

In [ ]:
calderdale_ip_tbl4 <- calderdale_ip_tbl3 %>%
    filter(
        start >= PARSE_DATE('%Y-%m-%d',start_date),
        end < PARSE_DATE('%Y-%m-%d',end_date)
    ) %>%
    inner_join(gp_person_tbl, by="person_id") %>%
    distinct()

In [ ]:
calderdale_ip_tbl4 %>% count() # 4612 v2 same

In [ ]:
calderdale_ip_df <- calderdale_ip_tbl2 %>% collect() %>% rename(hosp_age = dob)
calderdale_ip_df2 <- calderdale_ip_tbl4 %>% collect() %>% mutate(hosp_age = as.integer(hosp_age))

In [ ]:
calderdale_ip_df %>% glimpse()

In [ ]:
calderdale_ip_final_df <- bind_rows(calderdale_ip_df, calderdale_ip_df2) %>% distinct()

In [ ]:
calderdale_ip_final_df %>% count() # 14764 v2 14296

In [ ]:
calderdale_ip_final_df %>% glimpse()

In [ ]:
saveRDS(calderdale_ip_final_df, 'data/calderdale_ip_df.rds')

## Calderdale ED data

In [ ]:
calderdale_ed_tbl <- tbl(con, "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Calderdale_FDM_Format.tbl_SUS_Calderdale_EC_Backward_Compatible_20170701_To_20220731") %>% #glimpse()
    select(
        person_id,
        ed_age = age_at_start_of_episode,
        ed_sex = sex,
        tbl_SUS_Calderdale_EC_Backward_Compatible_20170701_To_20220731_start_date,
        tbl_SUS_Calderdale_EC_Backward_Compatible_20170701_To_20220731_end_date,
        starts_with('accident_and_emergency_investigation'), 
        starts_with('accident_and_emergency_treatment'), 
        starts_with('accident_and_emergency_diagnosis'),
        aande_attendance_category,
        aande_attendance_disposal
    ) %>%
    mutate(
        start = tbl_SUS_Calderdale_EC_Backward_Compatible_20170701_To_20220731_start_date,
        end = tbl_SUS_Calderdale_EC_Backward_Compatible_20170701_To_20220731_end_date
    )

In [ ]:
calderdale_ed_tbl %>% count(aande_attendance_disposal) %>% arrange(desc(n))

In [ ]:
calderdale_ed_tbl2 <- calderdale_ed_tbl %>%
    filter(
        start >= start_date,
        end < end_date
    ) %>%
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
calderdale_ed_tbl2 %>% count() # 11807 v2 same

In [ ]:
calderdale_ed_tbl3 <- tbl(con, "yhcr-prd-phm-bia-core.CB_MYSPACE_RP_CalderdaleSUS.src_SUS_Calderdale_EC_Backward_Compatible_CM_20220601_to_20230930") %>% #glimpse()
    select(
        person_id,
        ed_age = age_at_start_of_episode,
        ed_sex = sex,
        arrival_date,
        arrival_time,
        aande_departure_date,
        aande_departure_time,
        starts_with('accident_and_emergency_investigation'), 
        starts_with('accident_and_emergency_treatment'), 
        starts_with('accident_and_emergency_diagnosis'),
        aande_attendance_category,
        aande_attendance_disposal
    ) %>%
    inner_join(person_lsoa_tbl, by="person_id") %>%
    distinct()

In [ ]:
calderdale_ed_df <- calderdale_ed_tbl2 %>% collect() %>% select(-starts_with('tbl_SUS_Calderdale')) #%>% mutate(ed_age = as.integer(ed_age))

In [ ]:
calderdale_ed_dfa <- calderdale_ed_df %>% #head(n=100) %>%
    transmute(
        person_id,
        start,
        end,
        ed_sex = case_when(
            ed_sex == "1" ~ "male",
            ed_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        ed_age = as.integer(ed_age),
        avoid_invest = case_when(
            #if_all(starts_with('investigation'), ~ .x == 'NULL') ~ 1,
            if_all(starts_with('accident_and_emergency_investigation'), ~ is.na(.x) | (.x %in% avoidable_investigation_codes)) ~ 1,
            TRUE ~ 0
        ),
        avoid_treat = case_when(
            if_all(starts_with('accident_and_emergency_treatment'), ~ is.na(.x) | (.x %in% avoidable_treatment_codes)) ~ 1,
            TRUE ~ 0
        ),
        avoid_disp = case_when(
            # Based on Sue Croft consult, using revised version
            #attendance_disposal %in% avoidable_disposal_codes ~ 1,
            !aande_attendance_disposal %in% avoidable_disposal_codes ~ 1,
            TRUE ~ 0
        ),
        avoidable_admission = if_else(avoid_invest == 1 & avoid_treat == 1 & avoid_disp == 1, 1, 0),
        imd_rank,
        imd_decile,
        lsoa,
        birth_datetime,
        ethnicity_source_value
    )

In [ ]:
calderdale_ed_dfa %>% glimpse()


In [ ]:
calderdale_ed_df2 <- calderdale_ed_tbl3 %>% collect()

In [ ]:
# Fix new data dates

In [ ]:
calderdale_ed_df3 <- calderdale_ed_df2 %>% #glimpse()
    transmute(
        person_id,
        ed_age = as.integer(ed_age),
        ed_sex = case_when(
            ed_sex == "1" ~ "male",
            ed_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        avoid_invest = case_when(
            #if_all(starts_with('investigation'), ~ .x == 'NULL') ~ 1,
            if_all(starts_with('accident_and_emergency_investigation'), ~ is.na(.x) | (.x %in% avoidable_investigation_codes)) ~ 1,
            TRUE ~ 0
        ),
        avoid_treat = case_when(
            if_all(starts_with('accident_and_emergency_treatment'), ~ is.na(.x) | (.x %in% avoidable_treatment_codes)) ~ 1,
            TRUE ~ 0
        ),
        avoid_disp = case_when(
            # Based on Sue Croft consult, using revised version
            #attendance_disposal %in% avoidable_disposal_codes ~ 1,
            !aande_attendance_disposal %in% avoidable_disposal_codes ~ 1,
            TRUE ~ 0
        ),
        avoidable_admission = if_else(avoid_invest == 1 & avoid_treat == 1 & avoid_disp == 1, 1, 0) ,
        #start_dt = paste(arrival_date, arrival_time),
        #end_dt = paste(aande_departure_date, aande_departure_time),
        start = ymd_hm(paste(arrival_date, arrival_time)),
        end = ymd_hms(paste(aande_departure_date, aande_departure_time)),
        imd_rank,
        imd_decile,
        lsoa,
        birth_datetime,
        ethnicity_source_value
    ) %>%
    filter(
        start >= start_date,
        end < end_date
    )

In [ ]:
calderdale_ed_df3 %>% count() # 22806 v2 same

In [ ]:
calderdale_ed_df3 %>% glimpse()

In [ ]:
calderdale_ed_df3 %>% count(avoidable_admission) # v2 1- 1713

In [ ]:
calderdale_ed_df4 <- bind_rows(calderdale_ed_dfa, calderdale_ed_df3)

In [ ]:
calderdale_ed_df4 %>% glimpse()

In [ ]:
calderdale_ed_df4 %>% count(!is.na(end))

In [ ]:
saveRDS(calderdale_ed_df4, 'data/calderdale_ed_df.rds')

## Combine IP data

+ Will use hosp age and sex for IP

In [ ]:
#bri_ip_df <- readRDS('data/bri_ip_df.rds')
bri_ip_df <- readRDS('data/bri_ip_df.rds') %>% #glimpse()
    transmute(
        person_id,
        start = tbl_episode_start_date,
        end = tbl_episode_end_date,
        hosp_sex = case_when(
            hosp_sex == "1" ~ "male",
            hosp_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        hosp_age = as.integer(floor(as.numeric(difftime(tbl_episode_start_date, ymd(glue::glue("{birth_year}-{birth_month}-14")), units = "weeks"))/52.25)),
        hosp = 'BRI',
        lsoa,
        imd_decile,
        imd_rank,
        ethnicity_source_value
    )

# Sex 1 Male, 2 Female, anything else NA

In [ ]:
bri_ip_df %>% glimpse()

In [ ]:
airedale_ip_df <- readRDS('data/airedale_ip_df.rds') %>% #glimpse()
    transmute(
        person_id,
        start,
        end,
        hosp_sex = case_when(
            hosp_sex == "1" ~ "male",
            hosp_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        hosp_age = as.integer(floor(as.numeric(difftime(start, ymd(glue::glue("{dob}14")), units = "weeks"))/52.25)),
        hosp = 'Airedale',
        lsoa,
        imd_rank,
        imd_decile,
        ethnicity_source_value
    )

In [ ]:
airedale_ip_df %>% glimpse()

In [ ]:
calderdale_ip_df <- readRDS('data/calderdale_ip_df.rds') %>%
    transmute(
        person_id, 
        start,
        end,
        hosp_sex = case_when(
            hosp_sex == "1" ~ "male",
            hosp_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        hosp_age = bit64::as.integer.integer64(hosp_age),
        hosp = 'Calderdale',
        lsoa,
        imd_rank,
        imd_decile,
        ethnicity_source_value
    )

In [ ]:
calderdale_ip_df %>% glimpse()

In [ ]:
hosp_ip_df <- bind_rows(bri_ip_df, airedale_ip_df, calderdale_ip_df) %>% mutate(unique_id = glue::glue("HOSP_IP_{row_number()}"))

In [ ]:
hosp_ip_df %>% glimpse()

In [ ]:
saveRDS(hosp_ip_df, 'data/hosp_ip_df.rds')

## Combine ED data

In [ ]:
bri_ed_df <- readRDS('data/bri_ae_df.rds') %>% #glimpse() #%>%
    transmute(
        person_id, 
        start = tbl_ae_start_date,
        end = tbl_ae_end_date,
        avoidable_admission,
        hosp = 'BRI',
        ed_age = as.integer(hosp_age),
        ed_sex = case_when(
            hosp_sex == "1" ~ "male",
            hosp_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        lsoa,
        imd_rank,
        imd_decile,
        ethnicity_source_value
    )

In [ ]:
bri_ed_df %>% glimpse()

In [ ]:
airedale_ed_df <- readRDS('data/airedale_ed_df.rds') %>% #glimpse() %>%
    transmute(
        person_id,
        start,
        end,
        avoidable_admission,
        hosp = 'Airedale',
        ed_age = case_when(
         !is.na(hosp_age) ~ as.integer(hosp_age),
         #!is.na(dob) ~ as.integer(floor(as.numeric(difftime(start, ymd(glue::glue("{dob}-14")), units = "weeks"))/52.25)),
         !is.na(birth_datetime) ~ as.integer(difftime(start, birth_datetime, unit = "weeks")/52),
         TRUE ~ NA_integer_
        ),
        ed_sex = case_when(
            hosp_sex == "1" ~ "male",
            hosp_sex == "2" ~ "female",
            TRUE ~ NA_character_
        ),
        lsoa,
        imd_rank,
        imd_decile,
        ethnicity_source_value
    )

In [ ]:
airedale_ed_df %>% glimpse()

In [ ]:
calderdale_ed_df <- readRDS('data/calderdale_ed_df.rds') %>% #glimpse()
    transmute(
        person_id,
        start,
        end,
        avoidable_admission,
        hosp = 'Calderdale',
        ed_sex,
        ed_age,
        lsoa,
        imd_rank,
        imd_decile,
        ethnicity_source_value
    )

In [ ]:
calderdale_ed_df %>% glimpse()

In [ ]:
hosp_ed_df <- bind_rows(bri_ed_df, airedale_ed_df, calderdale_ed_df) %>% mutate(unique_id = glue::glue("HOSP_ED_{row_number()}"))

In [ ]:
hosp_ed_df %>% count(hosp) # Airedale 71311, BRI 177933, Calderdale 37414 v2 Airedale 76566, BRI 163319, Calderdale 34613

In [ ]:
hosp_ed_df %>% glimpse()

In [ ]:
hosp_ed_df %>% summarise(diff_time = mean(end-start)) # 293.6 minutes - round to 5 hours v2 about 4.8 hours so 5 reasonable

In [ ]:
# Amend the four cases where end time is less than start
hosp_ed_df <- hosp_ed_df %>%
 mutate(
   end = if_else(end <= start, start + hours(5), end)
 )

In [ ]:
saveRDS(hosp_ed_df, 'data/hosp_ed_df.rds')

In [ ]:
hosp_ed_df %>% count(hosp, avoidable_admission)